In [1]:
import os
import re
import shutil
import sys
from datetime import date
from copy import deepcopy
import json

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
from matplotlib.patches import Rectangle

# local imports
from image_arrays import *
from hdf_utils import pack_hdf, unpack_hdf
from fitting import BiexpFitter
from plot_utils import clean_axes

### Activate interactive plotting
By default, inline plots are static. Here we specify one of two options (comment out the undesired command) that will open plots with GUI controls for us.
- **qt ->** figures opened in windows outside the notebook
- **notebook ->** figures within notebook underneath generating cell.

In [3]:
%matplotlib notebook

In [32]:
base_path = "/mnt/Data/prerna_noise/"
data_path = os.path.join(base_path, "2022_05_24_rf_bar/")

with open(os.path.join(data_path, "conf.json")) as c:
    conf = json.load(c)
    
print("Configuration")
print("-------------")
for k, v in conf.items():
   print("%s:" % k, v) 

angles, traces, min_pts, max_pts = [], [], 1e10, 0
for f in os.listdir(data_path):
    if f.startswith("RF_bar_"):
        angles.append(int(f.lstrip("RF_bar_").rstrip(".csv")))
        traces.append(np.loadtxt(os.path.join(data_path, f), delimiter=",", skiprows=1).T)
        pts = traces[-1].shape[1]
        min_pts = min(pts, min_pts)
#         max_pts = max(pts, max_pts)
        
angles = np.array(angles)
offsets = np.array(conf["offsets"])
traces = np.stack([t[:, (t.shape[1] - min_pts):] for t in traces], axis=0)
n_angles, n_offsets, n_pts = traces.shape

print("\nData")
print("-------------")
print("angles:", angles, "\n")
print("offsets:", offsets, "\n")
print("traces shape (angles, offsets, pts):", traces.shape)

Configuration
-------------
offsets: [-300, -275, -250, -225, -200, -175, -150, -125, -100, -75, -50, -25, 0, 25, 50, 75, 100, 125, 150, 175, 200, 225, 250, 275, 300]

Data
-------------
angles: [  0 112 135 157  22  45  67  90] 

offsets: [-300 -275 -250 -225 -200 -175 -150 -125 -100  -75  -50  -25    0   25
   50   75  100  125  150  175  200  225  250  275  300] 

traces shape (angles, offsets, pts): (8, 25, 84)
